# Carbonate system through SeaCURE system 

### functions required for subsequent calculations

In [301]:
import numpy as np
import PyCO2SYS as pyco2
import pandas as pd

def carbon_chem_from_alk_and_dic(alkalinity, dic, temperature, salinity):
    # * **Total alkalinity** (type `1`) in μmol·kg<sup>−1</sup>.
    # * **Dissolved inorganic carbon** (type `2`) in μmol·kg<sup>−1</sup>.
    # * **pH** (type `3`) on the Total, Seawater, Free or NBS scale[^1].  Which scale is given by the input `pHSCALEIN`.
    # * **Partial pressure** (type `4`) or **fugacity** (type `5`) **of CO<sub>2</sub>** in μatm or **aqueous CO<sub>2</sub>** (type `8`) in μmol·kg<sup>−1</sup>.
    # * **Carbonate ion** (type `6`) in μmol·kg<sup>−1</sup>.
    # * **Bicarbonate ion** (type `7`) in μmol·kg<sup>−1</sup>.
    # Define input conditions
    kwargs_alk_dic = dict(
        par1 = alkalinity,  # Value of the first parameter
        par2 = dic,  # Value of the second parameter, which is a long vector of different DIC's!
        par1_type = 1,  # The first parameter supplied is of type "1", which is "alkalinity"
        par2_type = 2,  # The second parameter supplied is of type "2", which is "DIC"
        salinity = salinity,  # Salinity of the sample
        temperature = temperature,  # Temperature at input conditions
    )
    return pd.DataFrame.from_dict(pyco2.sys(**kwargs_alk_dic), orient='index').T

def carbon_chem_from_pH_and_dic(pH, dic, temperature, salinity):
    # Define input conditions
    kwargs_pH_dic = dict(
        par1 = dic,  # Value of the first parameter
        par2 = pH,  # Value of the second parameter, which is a long vector of different DIC's!
        par1_type = 2,  # The first parameter supplied is of type "1", which is "alkalinity"
        par2_type = 3,  # The second parameter supplied is of type "2", which is "DIC"
        salinity = salinity,  # Salinity of the sample
        temperature = temperature,  # Temperature at input conditions
    )
    return pd.DataFrame.from_dict(pyco2.sys(**kwargs_pH_dic), orient='index').T

def conservative_mixing(conc1,conc2,fraction_conc1):
    return (conc1 * fraction_conc1) + (conc2 * (1.0 - fraction_conc1))


def itterate_on_pH(co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph,increment,direction):
    counter = 0
    if direction == 'down':
        while co2_stripper_predicted_ph > pH_in_seawater_co2_stripper:
            initial_pH_estimate -= increment
            counter += 1
            ED_cell_seawater = carbon_chem_from_pH_and_dic(initial_pH_estimate, inflow_seawater['dic'].values, temperature, salinity)
            alkalinity_after_mixing_streams = conservative_mixing(ED_cell_seawater['alkalinity'].values,inflow_seawater['alkalinity'].values,fraction_of_water_passing_through_ED_acid_stream)
            co2_stripper_conditions = carbon_chem_from_alk_and_dic(alkalinity_after_mixing_streams, inflow_seawater_dissolved_inorganic_carbon, seawater_temperature, seawater_salinity)
            co2_stripper_predicted_ph = co2_stripper_conditions['pH'].values
            if counter > 10000:
                print('max iterations hit')
                break
    else:
        while co2_stripper_predicted_ph < pH_in_seawater_co2_stripper:
            initial_pH_estimate += increment
            counter += 1
            ED_cell_seawater = carbon_chem_from_pH_and_dic(initial_pH_estimate, inflow_seawater['dic'].values, temperature, salinity)
            alkalinity_after_mixing_streams = conservative_mixing(ED_cell_seawater['alkalinity'].values,inflow_seawater['alkalinity'].values,fraction_of_water_passing_through_ED_acid_stream)
            co2_stripper_conditions = carbon_chem_from_alk_and_dic(alkalinity_after_mixing_streams, inflow_seawater_dissolved_inorganic_carbon, seawater_temperature, seawater_salinity)
            co2_stripper_predicted_ph = co2_stripper_conditions['pH'].values
            if counter > 10000:
                print('max iterations hit')
                break
    return ED_cell_seawater,co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph

### Define inputs

In [334]:
seawater_temperature = 10.0 # degrees C
seawater_salinity = 35.0 # psu
inflow_seawater_alkalinity = 2300.0 # µeq/kg
inflow_seawater_dissolved_inorganic_carbon = 2100.0 # µmol/kg
fraction_of_water_passing_through_ED_acid_stream = 0.1 # decimal fraction of the incoming seawater stream diverted through the ED unit
pH_in_seawater_co2_stripper = 4.0
fraction_of_possible_co2_extracted_to_gas_stream = 1.0 # fraction of the available CO2 removed from teh seawater in the sewater Co2 stripper
fraction_of_water_passing_through_ED_base_stream = fraction_of_water_passing_through_ED_acid_stream # initial assumption is that the same volume of fluid is passing through the ED cell's base stream as through the acid stream
fraction_of_co2_depleted_water_in_final_outflow = 0.25 # i.e. how much unpricessed seawater if being mixed with the processed water when returned back to the ocean

### Chemistry of inflow seawater

Note, initial output variables are:

pH - self explanatory

dic - dissolved inorganic carbon concentration. i.e. total amount of dissolved carbon in the seawater in µmol/kg

alkalinity - total alkalinity, 'the molar amount of bases in a solution that can be converted to uncharged species by a strong acid' ([wikipedia](https://en.wikipedia.org/wiki/Alkalinity#cite_note-2)), µeq/kg

bicarbonate - concentration of bicarbonate ions µmol/kg

carbonate - concentration of carbonate ions µmol/kg

pCO2 - partial presure of CO2, µatm

In [335]:
inflow_seawater = carbon_chem_from_alk_and_dic(inflow_seawater_alkalinity, inflow_seawater_dissolved_inorganic_carbon, seawater_temperature, seawater_salinity)
inflow_seawater[['pH','dic','alkalinity','bicarbonate', 'carbonate','pCO2']]


,pH,dic,alkalinity,bicarbonate,carbonate,pCO2
0,8.091704847930695,2100.0,2300.0,1940.980368,143.206398,361.773084


### Conditions in acid stream of ED cell

Crude itteration to find pH in ED cell consistent with the required seawater CO2 stripper pH and fraction of seawaterpassed trhough ED cell

In [336]:
counter = 0
co2_stripper_predicted_ph = 1000.0
initial_pH_estimate = pH_in_seawater_co2_stripper
co2_stripper_conditions = 0.0

ED_cell_seawater,co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph = itterate_on_pH(co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph,0.1,'down')
ED_cell_seawater,co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph = itterate_on_pH(co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph,0.01,'up')
ED_cell_seawater,co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph = itterate_on_pH(co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph,0.001,'down')
ED_cell_seawater,co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph = itterate_on_pH(co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph,0.0001,'up')
ED_cell_seawater,co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph = itterate_on_pH(co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph,0.00001,'down')
ED_cell_seawater,co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph = itterate_on_pH(co2_stripper_conditions,initial_pH_estimate,co2_stripper_predicted_ph,0.000001,'up')

ED_cell_seawater[['pH','dic','alkalinity','bicarbonate', 'carbonate','pCO2']]


,pH,dic,alkalinity,bicarbonate,carbonate,pCO2
0,1.662965999999998,2100.0,-21511.534919,0.096042,0.0,48041.328496


### Conditions in seawater CO2 stripper

In [337]:
co2_stripper_conditions[['pH','dic','alkalinity','bicarbonate', 'carbonate','pCO2']]

,pH,dic,alkalinity,bicarbonate,carbonate,pCO2
0,4.000006046230498,2100.0,-81.153492,20.664562,0.000123,47570.761768


### Fraction of dissolved carbon potentially liberated

In [338]:
fraction_of_liberated_dic = (inflow_seawater_dissolved_inorganic_carbon - co2_stripper_conditions['bicarbonate'].values - co2_stripper_conditions['carbonate'].values)/inflow_seawater_dissolved_inorganic_carbon
print(fraction_of_liberated_dic)

[0.9901596738254074]


### Conditions in base stream of ED cell


In [339]:
remaining_dic = (co2_stripper_conditions['dic'].values * (1.0 - fraction_of_liberated_dic)) * (1.0/fraction_of_possible_co2_extracted_to_gas_stream)

ED_cell_base_seawater = carbon_chem_from_alk_and_dic(inflow_seawater_alkalinity - ED_cell_seawater['alkalinity'], remaining_dic, seawater_temperature, seawater_salinity)
ED_cell_base_seawater[['pH','dic','alkalinity','bicarbonate', 'carbonate','pCO2']]

,pH,dic,alkalinity,bicarbonate,carbonate,pCO2
0,12.210527851437211,20.664685,23811.534919,0.021282,20.643403,0.0


### Conditions after mixing base stream of ED cell with water from seawater CO2 stripper


In [340]:
alkalinity_after_ED_base_and_stripper_mixing = conservative_mixing(ED_cell_base_seawater['alkalinity'].values,co2_stripper_conditions['alkalinity'].values,fraction_of_water_passing_through_ED_base_stream)
after_ED_base_and_stripper_mixing = carbon_chem_from_alk_and_dic(alkalinity_after_ED_base_and_stripper_mixing, remaining_dic, seawater_temperature, seawater_salinity)
after_ED_base_and_stripper_mixing[['pH','dic','alkalinity','bicarbonate', 'carbonate','pCO2']]

,pH,dic,alkalinity,bicarbonate,carbonate,pCO2
0,11.110095387236399,20.664685,2308.115349,0.265028,20.399655,0.000047


### Conditions after mixing CO2 depleted water with natural seeawater in final outflow

In [341]:
alkalinity_in_outflow = conservative_mixing(alkalinity_after_ED_base_and_stripper_mixing,inflow_seawater_alkalinity,fraction_of_co2_depleted_water_in_final_outflow)
dic_in_outflow = conservative_mixing(remaining_dic,inflow_seawater_dissolved_inorganic_carbon,fraction_of_co2_depleted_water_in_final_outflow)

after_ED_base_and_stripper_mixing = carbon_chem_from_alk_and_dic(alkalinity_in_outflow, dic_in_outflow, seawater_temperature, seawater_salinity)
after_ED_base_and_stripper_mixing[['pH','dic','alkalinity','bicarbonate', 'carbonate','pCO2']]

,pH,dic,alkalinity,bicarbonate,carbonate,pCO2
0,8.86868693539054,1580.166171,2302.028837,1095.167031,483.508071,34.112487


ADD a diagram of the system with the numbers popping up in the right oplace fopr the specified variable